In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt


In [2]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081))
])

In [3]:
train_dataset = datasets.MNIST(
    root='data', train=True, download=True, transform=transform)
test_dataset = datasets.MNIST(
    root='data', train=False, download=True, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=1000)


Failed to download (trying next):
HTTP Error 404: Not Found



100%|██████████| 9.91M/9.91M [00:01<00:00, 9.45MB/s]


Extracting data\MNIST\raw\train-images-idx3-ubyte.gz to data\MNIST\raw

Failed to download (trying next):
HTTP Error 404: Not Found



100%|██████████| 28.9k/28.9k [00:00<00:00, 265kB/s]


Extracting data\MNIST\raw\train-labels-idx1-ubyte.gz to data\MNIST\raw

Failed to download (trying next):
HTTP Error 404: Not Found



100%|██████████| 1.65M/1.65M [00:00<00:00, 2.48MB/s]


Extracting data\MNIST\raw\t10k-images-idx3-ubyte.gz to data\MNIST\raw

Failed to download (trying next):
HTTP Error 404: Not Found



100%|██████████| 4.54k/4.54k [00:00<00:00, 4.54MB/s]

Extracting data\MNIST\raw\t10k-labels-idx1-ubyte.gz to data\MNIST\raw



In [5]:
train_dataset

Dataset MNIST
    Number of datapoints: 60000
    Root location: data
    Split: Train
    StandardTransform
Transform: Compose(
               ToTensor()
               Normalize(mean=(0.1307,), std=0.3081)
           )

In [6]:
model = nn.Sequential(
    nn.Flatten(),             # 28x28 → 784
    nn.Linear(784, 128),      # warstwa ukryta
    nn.ReLU(),                # funkcja aktywacji
    nn.Linear(128, 10)        # warstwa wyjściowa – 10 klas
)


In [7]:
loss_fn = nn.CrossEntropyLoss()


In [8]:
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)


In [9]:
def train(model, loader, loss_fn, optimizer, epoch):
    model.train()  #tryb treningu
    total_loss = 0

    for batch_idx, (data, target) in enumerate(loader):
        optimizer.zero_grad()             # zerujemy gradienty
        output = model(data)              # forward pass
        loss = loss_fn(output, target)    # obliczamy stratę
        loss.backward()                   # backpropagation
        optimizer.step()                  # aktualizacja wag

        total_loss += loss.item()
        if batch_idx % 100 == 0:
            print(f"Epoch {epoch} | Batch {batch_idx} | Loss: {loss.item():.4f}")

    avg_loss = total_loss / len(loader)
    print(f">>> Epoch {epoch} | Avg Loss: {avg_loss:.4f}")


In [10]:
def test(model, loader, loss_fn):
    model.eval()
    test_loss = 0
    correct = 0

    with torch.no_grad():  # wyłączamy gradienty – oszczędność zasobów
        for data, target in loader:
            output = model(data)
            test_loss += loss_fn(output, target).item()
            pred = output.argmax(dim=1)  # indeks największej wartości
            correct += (pred == target).sum().item()

    test_loss /= len(loader)
    accuracy = correct / len(loader.dataset)
    print(f"Test Loss: {test_loss:.4f}, Accuracy: {accuracy:.4f}")


In [11]:
epochs = 5
for epoch in range(1, epochs + 1):
    train(model, train_loader, loss_fn, optimizer, epoch)
    test(model, test_loader, loss_fn)


Epoch 1 | Batch 0 | Loss: 2.3008
Epoch 1 | Batch 100 | Loss: 0.4626
Epoch 1 | Batch 200 | Loss: 0.3536
Epoch 1 | Batch 300 | Loss: 0.1661
Epoch 1 | Batch 400 | Loss: 0.2145
Epoch 1 | Batch 500 | Loss: 0.0872
Epoch 1 | Batch 600 | Loss: 0.1453
Epoch 1 | Batch 700 | Loss: 0.2373
Epoch 1 | Batch 800 | Loss: 0.0446
Epoch 1 | Batch 900 | Loss: 0.1248
>>> Epoch 1 | Avg Loss: 0.2774
Test Loss: 0.1491, Accuracy: 0.9550
Epoch 2 | Batch 0 | Loss: 0.0657
Epoch 2 | Batch 100 | Loss: 0.1306
Epoch 2 | Batch 200 | Loss: 0.1597
Epoch 2 | Batch 300 | Loss: 0.0542
Epoch 2 | Batch 400 | Loss: 0.1842
Epoch 2 | Batch 500 | Loss: 0.0743
Epoch 2 | Batch 600 | Loss: 0.0517
Epoch 2 | Batch 700 | Loss: 0.1143
Epoch 2 | Batch 800 | Loss: 0.1606
Epoch 2 | Batch 900 | Loss: 0.1508
>>> Epoch 2 | Avg Loss: 0.1162
Test Loss: 0.0991, Accuracy: 0.9704
Epoch 3 | Batch 0 | Loss: 0.0549
Epoch 3 | Batch 100 | Loss: 0.0660
Epoch 3 | Batch 200 | Loss: 0.0367
Epoch 3 | Batch 300 | Loss: 0.2653
Epoch 3 | Batch 400 | Loss: 0.08